In [ ]:
using DynamicPolynomials
using TSSOS
using MAT
using LinearAlgebra
using Printf

using JuMP

In [2]:
@polyvar temp1[1:4, 1:4]
@polyvar temp2[1:4, 1:4]
temp_p = temp2 * 1.1 + temp1 * 1.1

function skew(q)
    qx = temp_p[1:3, 1:3] * 0
    qx[1, 2] = -q[3]
    qx[2, 1] =  q[3]

    qx[1, 3] =  q[2]
    qx[3, 1] = -q[2]

    qx[2, 3] = -q[1]
    qx[3, 2] =  q[1]

    return qx
end

function LeftMultiplication(q)
    Lq = temp_p * 0
    Lq[1, 1]   =  q[1]
    Lq[1, 2:4] = -q[2:4]
    Lq[2:4, 1] = q[2:4]
    Lq[2:4, 2:4] = q[1] * Matrix(1.0I, 3, 3) + skew(q[2:4])
    return Lq
end

function RightMultiplication(q)
    Rq = temp_p * 0
    Rq[1, 1]   =  q[1]
    Rq[1, 2:4] = -q[2:4]
    Rq[2:4, 1] = q[2:4]
    Rq[2:4, 2:4] = q[1] * Matrix(1.0I, 3, 3) - skew(q[2:4])
    return Rq
end

RightMultiplication (generic function with 1 method)

In [3]:
V = temp_p[1:3, :] * 0
V[1:3, 2:4] = Matrix(1.0I, 3, 3)
T = temp_p * 0
T[2:4, 2:4] = -Matrix(1.0I, 3, 3)
T

Jb = Matrix(1.0I, 3, 3)
Jb[1,1] = 1.0 / 10
Jb[2,2] = 2.0 / 10
Jb[3,3] = 1.0 / 10
m = 0.5

G = temp_p[1:3, 1] * 0
G[3] = 1

1

In [4]:
fileIn = matopen("freespace.mat")
dset = read(fileIn)
close(fileIn)
A_list = dset["A_list"]
b_list = dset["b_list"]
e_list = dset["e_in"]
# 1 2 3 4 5 7 10 11 12

13×2 Matrix{Float64}:
  1.0   2.0
  2.0   3.0
  3.0   4.0
  3.0   7.0
  3.0   6.0
  4.0   5.0
  5.0   7.0
  6.0   8.0
  8.0   9.0
  9.0  10.0
  7.0  10.0
 10.0  11.0
 11.0  12.0

In [ ]:
# Rotation + position with scale 
# Safe corridor WITH connector
N = 80
dt = 0.125

@polyvar q[1:N+1, 1:4]
@polyvar w[1:N+1, 1:4]

@polyvar p[1:N+1, 1:3]
@polyvar v[1:N+1, 1:3]

@polyvar p_load[1:N+1, 1:3]
@polyvar v_load[1:N+1, 1:3]
@polyvar lam[1:N]

@polyvar tau[1:N, 1:3]
@polyvar f[1:N, 1:3]
@polyvar r[1:N]

cable_len = 1.0 / 2
m_load = 0.5

var = append!(reshape(q, 4 * (N+1)), reshape(w, 4*(N+1)), 
              reshape(p, 3 * (N+1)), reshape(v, 3*(N+1)), 
              reshape(p_load, 3 * (N+1)), reshape(v_load, 3*(N+1)), reshape(lam, N),
              reshape(tau, 3*N), reshape(f, 3 * N), r)

gravity = temp_p[1:3, 1] * 0
gravity[1] = 0.0
gravity[2] = 0.0
gravity[3] = 9.81

eq = [temp_p[1,1] * 0]
ineq = []


quat_list = [[1;0;0;0], [0.86;0;0.5;0], [1;0;1;0], [0.5, 0, 0.85, 0], [0,0,1,0]]
quat_exec = quat_list[1]
quat_exec = quat_exec / norm(quat_exec)

eq[1] = q[1, 1] - quat_exec[1]
append!(eq, [q[1, 2] - quat_exec[2]])
append!(eq, [q[1, 3] - quat_exec[3]])
append!(eq, [q[1, 4] - quat_exec[4]])

append!(eq, [w[1, 1] - 2 / dt])
append!(eq, [w[1, 2] - 0.0])
append!(eq, [w[1, 3] - 0.0])
append!(eq, [w[1, 4] - 0.0])

x_init = 0.2
y_init = 0.2

append!(eq, [p[1, 1] - x_init])
append!(eq, [p[1, 2] - y_init])
append!(eq, [p[1, 3] - 0.0])

append!(eq, [v[1, 1] - 0.0])
append!(eq, [v[1, 2] - 0.0])
append!(eq, [v[1, 3] - 0.0])

append!(eq, [p_load[1, 1] - x_init])
append!(eq, [p_load[1, 2] - y_init])
append!(eq, [p_load[1, 3] - (0 - cable_len)])

append!(eq, [v_load[1, 1] - 0.0])
append!(eq, [v_load[1, 2] - 0.0])
append!(eq, [v_load[1, 3] - 0.0])

Cost = 0
for k = 1:N

    qk   = q[k, 1:4]
    qkp1 = q[k+1, 1:4]
    wk   = w[k, 1:4]
    wkp1 = w[k+1, 1:4]
    tauk = tau[k, 1:3]

    pk   = p[k, 1:3]
    pkp1 = p[k+1, 1:3]
    vk   = v[k, 1:3]
    vkp1 = v[k+1, 1:3]
    fk   = f[k, 1:3]

    p_load_k   = p_load[  k, 1:3]
    p_load_kp1 = p_load[k+1, 1:3]
    v_load_k   = v_load[  k, 1:3]
    v_load_kp1 = v_load[k+1, 1:3] 

    # 1 2 3 4 5 7 10 11 12

    if k == 11 || k == 17 || k == 23 || k == 29 || k == 40 || k == 51 || k == 62 || k == 68
        # append!(eq, pkp1 - pk)
        # append!(eq, p_load_k - p_load_kp1)

        append!(eq, pk - pkp1)
        append!(eq, p_load_k - p_load_kp1)
        append!(eq, vk - vkp1)
        append!(eq, v_load_k - v_load_kp1)
        append!(eq, qk - qkp1)
        append!(eq, wk - wkp1)
        println("connect")
    else
        Mk   = Jb * wk[1] * wk[2:4] + skew(wk[2:4]) * Jb * wk[2:4]
        Mkp1 = Jb * wkp1[1] * wkp1[2:4] + skew(wkp1[2:4]) * Jb * wkp1[2:4]
    
        append!(eq, Mkp1 - Mk - 2 * tauk)
        append!(eq, qkp1 - dt / 2 * LeftMultiplication(qk) * wk)
        
        append!(ineq, [wkp1[1]])
        append!(ineq, [qkp1[1]])

        
        vec = pkp1 - p_load_kp1
        append!(eq, (m_load * (v_load_kp1 - v_load_k) / dt + m_load * gravity) - vec * lam[k])
        append!(eq, p_load_kp1 - (p_load_k + v_load_k * dt) )
        append!(eq, [vec[1]^2 + vec[2]^2 + vec[3]^2 - cable_len^2])

        append!(ineq, [lam[k] - 0.5])

        append!(ineq, [pkp1[3] + 0.2])
        append!(ineq, [- pkp1[3] + 0.2])
        append!(ineq, [- p_load_kp1[3] - cable_len + 0.2])
        append!(ineq, [  p_load_kp1[3] + cable_len + 0.2])
        # append!(ineq, [- p_load_kp1[3] - 0.8])
        # append!(ineq, [  p_load_kp1[3] + 1.2])

        vb = temp_p[1:3, 1] * 0
        vb[3] = 1.0
        Q  = V * RightMultiplication(qkp1)' * LeftMultiplication(qkp1) * V'
        vb = Q * vb

        append!(eq, (m * (vkp1 - vk) / dt + m * gravity) - fk + vec * lam[k])
        append!(eq, vb  - r[k] * fk)
        append!(eq, pkp1 - (pk + vk * dt) )
    end

    append!(eq, [qkp1[1]^2 + qkp1[2]^2 + qkp1[3]^2 + qkp1[4]^2 - 1])
    append!(eq, [(wkp1[1]^2 + wkp1[2]^2 + wkp1[3]^2 + wkp1[4]^2) - 4 / dt^2])


    
    if k < 11
        # v1
        append!(ineq, b_list[1] - A_list[1] * p_load_kp1[1:2])
        append!(ineq, b_list[1] - A_list[1] * pkp1[1:2])
    elseif 11 <= k < 17
        # v2
        append!(ineq, b_list[2] - A_list[2] * p_load_kp1[1:2])
        append!(ineq, b_list[2] - A_list[2] * pkp1[1:2])
    elseif 17 <= k < 23
        # v3
        append!(ineq, b_list[3] - A_list[3] * p_load_kp1[1:2])
        append!(ineq, b_list[3] - A_list[3] * pkp1[1:2])
    elseif 23 <= k < 29
        # v4
        append!(ineq, b_list[4] - A_list[4] * p_load_kp1[1:2])
        append!(ineq, b_list[4] - A_list[4] * pkp1[1:2])
    elseif 29 <= k < 40
        append!(ineq, b_list[5] - A_list[5] * p_load_kp1[1:2])
        append!(ineq, b_list[5] - A_list[5] * pkp1[1:2])
    elseif 40 <= k < 51
        append!(ineq, b_list[7] - A_list[7] * p_load_kp1[1:2])
        append!(ineq, b_list[7] - A_list[7] * pkp1[1:2])
    elseif 51 <= k < 62
        append!(ineq, b_list[10] - A_list[10] * p_load_kp1[1:2])
        append!(ineq, b_list[10] - A_list[10] * pkp1[1:2])
    elseif 62 <= k < 68
        append!(ineq, b_list[11] - A_list[11] * p_load_kp1[1:2])
        append!(ineq, b_list[11] - A_list[11] * pkp1[1:2])
    elseif 68 <= k < 80
        append!(ineq, b_list[12] - A_list[12] * p_load_kp1[1:2])
        append!(ineq, b_list[12] - A_list[12] * pkp1[1:2])
    end

    # Cost = Cost + (p_load_k[1]^2 + p_load_k[2]^2 + (p_load_k[3] + cable_len)^2)

    Cost = Cost + ((p_load_k[3] + cable_len)^2) * 10
    Cost = Cost + (p_load_k[1] - p_load_kp1[1])^2 + (p_load_k[2] - p_load_kp1[2])^2

    # Cost = Cost + (v_load_k[1]^2 + v_load_k[2]^2 + v_load_k[3]^2)
    Cost = Cost + (v_load_k - v_load_kp1)' * (v_load_k - v_load_kp1)
    Cost = Cost + lam[k]^2 * 0.01

    Cost = Cost + (tauk[1]^2 + tauk[2]^2 + tauk[3]^2) * 0.1
    Cost = Cost + (fk[1]^2 + fk[2]^2 + fk[3]^2) * 0.1
    Cost = Cost + (r[k]^2) * 0.01

    Cost = Cost + ((wk[1] - 1)^2 + wk[2]^2 + wk[3]^2 + wk[4]^2) * 10
    Cost = Cost + ((qk[1] - 1)^2 + qk[2]^2 + qk[3]^2 + qk[4]^2) * 0.1

    # Cost = Cost + (vk[1]^2 + vk[2]^2 + vk[3]^2) * 1
    Cost = Cost + (vk - vkp1)' * (vk - vkp1)
    # Cost = Cost + (pk[1]^2 + pk[2]^2 + pk[3]^2) * 0.1
    Cost = Cost + pk[3]^2 * 10
    Cost = Cost + (pk[1] - pkp1[1])^2 +  (pk[2] - pkp1[2])^2

end

qkp1 = q[end, :]
wkp1 = w[end, :]
pkp1 = p[end, :]
vkp1 = v[end, :]


Cost = Cost + ((wkp1[1] - 1)^2 + wkp1[2]^2 + wkp1[3]^2 + wkp1[4]^2) * 10
Cost = Cost + ((qkp1[1] - 1)^2 + qkp1[2]^2 + qkp1[3]^2 + qkp1[4]^2) * 100

Cost = Cost + (vkp1[1]^2 + vkp1[2]^2 + vkp1[3]^2) * 100
Cost = Cost + ((pkp1[1] - 4.8)^2 + (pkp1[2] - 4.8)^2 + pkp1[3]^2) * 100

p_load_kp1 = p_load[end, :]
v_load_kp1 = v_load[end, :]
Cost = Cost + (v_load_kp1[1]^2 + v_load_kp1[2]^2 + v_load_kp1[3]^2) * 100
Cost = Cost + ((p_load_kp1[1] - 5)^2 + (p_load_kp1[2] - 5)^2 + (p_load_kp1[3] + 1)^2) * 100

pop = append!([Cost], ineq, eq)
eq_len = length(eq)

order = 1

time_start = time()
## TSL note: Use the original PAI for your own applications! 
opt,sol,data =cs_tssos_first(pop, var, order, numeq=eq_len, CS="MF", TS = "MD", solution=true, QUIET=false, MomentOne=false)
# opt,sol,data,model,sol_approx =cs_tssos_first_export_tsl(pop, var, order, numeq=eq_len, CS="MF", TS = false, solution=true, QUIET=false, MomentOne=false)
time_end = time()

elapsed = time_end - time_start

moment = []
for k = 1:length(data.Mmatrix)
    append!(moment, [convert(Matrix{Float64}, data.Mmatrix[k])]) # data.Mmatrix[k])
end

logger = Dict("sol" => sol, "sol_approx" => sol_approx,
             "Ns" => N, "moment" => moment, "dt" => dt,
            "opt" => opt, "elapsed" => elapsed)
            # "primal" => flag, "termination" => flag2)


matwrite("logger_3.mat", logger)

connect
connect
connect
connect
connect
connect
connect
connect
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[7, 6, 5, 4, 3, 2, 1]
[126, 2, 380, 8, 140, 700, 298]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.4367673 seconds. The maximal size of cliques is 7.
Assembling the SDP...
There are 8914 affine constraints.
SDP assembling time: 0.8599716 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 8914            
  Cones                  : 0               
  Scalar variables       : 3133            
  Matrix variables       : 1654            
  Integer v